# Topic Modeling (Latent Dirichlet Allocation - LDA)

In [15]:
import pandas as pd
import gensim
import gensim.corpora as corpora
import pyLDAvis.gensim_models
import pyLDAvis
import pickle
import numpy as np

### Load data

In [2]:
clean_data = pd.read_csv("../data/preprocessed/bumble_preprocessed.csv")
clean_data['at'] = pd.to_datetime(clean_data['at'] )
clean_data['score'] = pd.to_numeric(clean_data['score'] )
clean_data.head(3)

,at,score,content,clean_content,sentiment
0,2022-03-28 23:33:04,5,Bumble rocks 👍🤞,bumble rock,0.000
1,2022-03-28 23:23:30,1,Just a cash grab. Congrats you have 2 new like...,cash grab congrats new like let swipe right fi...,0.071
2,2022-03-28 23:10:12,1,"Terrible, l have lost total faith in this app,...",terrible lost total faith app promised contact...,-0.118


### Build model

In [3]:
# Words to remove
extra_stopwords = ['app', 'bumble', 'account', 'profile', 'swipe', 'people']

# Create Dictionary
data_words = list(map(lambda x: [w for w in x.split() if w not in extra_stopwords], clean_data['clean_content']))
id2word = corpora.Dictionary(data_words) 

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_words]

In [53]:
# Build LDA model
num_topics = 6
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# # Save model
model_path = "./ldaModel_topics-"+str(num_topics)
with open(model_path, 'wb') as f:
    pickle.dump(lda_model, f)

In [54]:
# Prepare visualization

LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
model_path = "./ldaModelvis_topics-"+str(num_topics)
with open(model_path, 'wb') as f:
    pickle.dump(LDAvis_prepared, f)


pyLDAvis.enable_notebook()
LDAvis_prepared

C:\Users\bpalg\anaconda3\envs\swd\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.004824  0.021356       1        1  21.607384
4      0.024106 -0.033036       2        1  19.780346
1      0.013281 -0.039152       3        1  19.213214
0      0.016669 -0.015356       4        1  16.861245
5      0.037381  0.058670       5        1  12.540520
3     -0.096262  0.007520       6        1   9.997290, topic_info=         Term          Freq         Total Category  logprob  loglift
75       good  12280.000000  12280.000000  Default  30.0000  30.0000
71      great   8457.000000   8457.000000  Default  29.0000  29.0000
81     people  14120.000000  14120.000000  Default  28.0000  28.0000
131      nice   3878.000000   3878.000000  Default  27.0000  27.0000
594  facebook   3066.000000   3066.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
6         let    484.262116   3136.391236   Topic6  -5.4501   0.4347
241      want    555.941391   5081.744915   Topic6  -5.3121   0.0901
7        like    612.203506  16457.081466   Topic6  -5.2157  -0.9886
86      still    484.859756   4332.682556   Topic6  -5.4489   0.1128
97        pay    496.100840   6737.921899   Topic6  -5.4260  -0.3059

[554 rows x 6 columns], token_table=       Topic      Freq            Term
term                                  
9901       6  0.846364      acceptance
6961       4  0.899497             ace
10149      4  0.874848  administrative
5569       1  0.893843   aesthetically
4272       1  0.119090           aight
...      ...       ...             ...
614        2  0.052476             yet
614        3  0.149796             yet
614        4  0.100182             yet
614        5  0.130714             yet
614        6  0.123081             yet

[1540 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 2, 1, 6, 4])

### Performance metrics

In [55]:
coherence = gensim.models.CoherenceModel(model=lda_model, corpus=corpus, dictionary=id2word, coherence='u_mass').get_coherence()
perplexity = lda_model.log_perplexity(corpus)
print(f"Coherence: {coherence}")
print(f"Perplexity: {perplexity}")

### Get top words for each topic

In [50]:
# Choose number of words
num_words = 30

In [51]:
# Order to match visualization
order = pyLDAvis.PreparedData.to_dict(LDAvis_prepared)['topic.order']
topD = pd.DataFrame({'old': order, 'new': range(1,len(order)+1)})
topD.sort_values(by=['old'], inplace = True)

num_topics =lda_model.num_topics
a = lda_model.show_topics(num_topics=num_topics,formatted=False,num_words=num_words)
b = lda_model.top_topics(corpus,dictionary=id2word,topn=num_words) # This orders the topics in the decreasing order of coherence score

topic2skillb = {}
topic2csb = {}
topic2skilla = {}
topic2csa = {}

cnt =1

for ws in b:
    wset = set(w[1] for w in ws[0])
    topic2skillb[cnt] = wset
    topic2csb[cnt] = ws[1]
    cnt +=1

for ws in a:
    wset = set(w[0]for w in ws[1])
    topic2skilla[ws[0]+1] = wset

for i in range(1,num_topics+1):
    j = 1
    cont = True
    while j < num_topics+1 and cont:
        if topic2skillb[j]==topic2skilla[i]:
            topic2csa[i] = topic2csb[j]
            cont = False
        j+=1
        

finalData = pd.DataFrame([],columns=['Topic','words'])
finalData['Topic']=topD['new']
finalData['Topic'] = finalData['Topic'].apply(lambda x: 'Topic'+str(x))
finalData['words']=topic2skilla.values()
finalData['cs'] = topic2csa.values()
finalData.sort_values(by='cs',ascending=False,inplace=True)

In [52]:
finalData

,Topic,words,cs
2,Topic3,"{swipe, people, even, see, pay, easy, tinder, ...",-2.608418
1,Topic2,"{fake, swipe, people, month, getting, money, e...",-2.634449
0,Topic1,"{fake, really, people, see, looking, girl, tin...",-2.697719
3,Topic4,"{fake, people, money, many, good, way, make, e...",-2.796193
6,Topic7,"{people, money, even, customer, easy, facebook...",-2.862784
4,Topic5,"{really, conversation, subscription, upload, p...",-2.865669
5,Topic6,"{subscription, people, getting, month, money, ...",-2.935792
7,Topic8,"{really, swipe, people, getting, filter, see, ...",-3.019370


### Get representative reviews for each topic

In [20]:
# Document - Topic probability matrix
lda_output = lda_model[corpus]
df_document_topic = pd.DataFrame(np.zeros((len(clean_data),lda_model.num_topics)))
df_document_topic.columns = df_document_topic.columns+1
c = 0
for o in lda_output:
    idx = list(map(lambda x: int(topD.loc[topD['old']==x[0]+1,'new']), o))
    val = list(map(lambda x: round(x[1],3), o))
    df_document_topic.loc[c,idx] = val
    c+=1

In [23]:
# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
max_proba = np.max(df_document_topic.values, axis = 1)

# Styling
def color_green(val):
    color = 'green' if val > .15 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .15 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)

,1,2,3,4,5
0,0.101000,0.102000,0.595000,0.102000,0.100000
1,0.014000,0.014000,0.945000,0.014000,0.014000
2,0.013000,0.013000,0.013000,0.949000,0.013000
3,0.067000,0.731000,0.067000,0.068000,0.068000
4,0.012000,0.012000,0.012000,0.204000,0.760000
5,0.897000,0.026000,0.025000,0.025000,0.026000
6,0.000000,0.000000,0.000000,0.965000,0.000000
7,0.067000,0.731000,0.067000,0.068000,0.068000
8,0.040000,0.040000,0.040000,0.839000,0.040000
9,0.834000,0.041000,0.041000,0.042000,0.041000


In [42]:
rep_docs_lda = pd.DataFrame()
n_examples = 3

for c in df_document_topic.columns:
    aux = df_document_topic[c].copy()
    aux.sort_values(ascending=False, inplace = True)
    ix = aux.index.values
    ix = ix[:n_examples]
    rep_docs_lda[c] = clean_data.loc[ix,'clean_content'].values
    print("Topic ", c)
    for i in np.arange(n_examples):
        print(rep_docs_lda[c][i], end="\n\n")

Topic  1
concept fine doubt many profile actually active sure everyone look good mean much actively using app swipe right profile every day lucky get match every couple week time come empty even get match time either miss due time window ignore unmatch disappear within hour two wish exaggerating actual conversation match year multiple photo different angle including full body shot witty profile verified real reason getting least odd match like concept disappointed execution seems might better tinder least get odd match actually interact people time

decided give app another shot longer required seems like bumble stack match popular see attractive woman first see compare tinder without question swipping day single match barely even match queue even tell keep thinking swipe enough baithing suit super model perfect body might actually make real people looking grim cost month also insane mean crazy app supposed match someone like user pay way le subscription imbue confidence find someone m